# Import

In [1]:
import csv
import os
import sys

import numpy as np
import pandas as pd
from nltk import word_tokenize
from stop_words import get_stop_words
from nltk.corpus import stopwords
from tqdm import tqdm

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


## Add configuration file

In [12]:
sys.path.append("/home/jovyan/core/config/")

In [3]:
from ALL import config 

## Set condition

In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

# Read data

In [5]:
ag_news_df = pd.read_csv("../../DataShaping/data/AgNews/master.csv", index_col=0)

In [6]:
with open("../../DataShaping/data/AgNews/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

# Word tokinize

In [7]:
ag_news_df["words"] = ag_news_df.text.progress_apply(word_tokenize)

100%|██████████| 120000/120000 [08:11<00:00, 244.12it/s]


In [8]:
stop_words_add = ["would", "could", "should"]
stop_char = ["==", "--", "\'s", "''", "n't", "``","..", "...", "....", "'m", "'ve", "'d", "'ll", ""]
stop_words = set(stopwords.words("english") + get_stop_words("english") + stop_words_add + stop_char)

In [9]:
#     一文字以下の単語とstop_word, stop_charを削除
ag_news_df["words_nonstop"] = ag_news_df.words.progress_apply(
    lambda words: [
        word for word in words if word.lower() not in stop_words if len(word)> 1
    ]
)

100%|██████████| 120000/120000 [00:08<00:00, 13630.65it/s]


In [10]:
ag_news_df.words = ag_news_df.words.progress_apply(lambda words: " ".join(words))
ag_news_df.words_nonstop = ag_news_df.words_nonstop.progress_apply(
    lambda words: " ".join(words)
)

100%|██████████| 120000/120000 [00:02<00:00, 44936.91it/s]


In [11]:
file_path = "../data/AgNews"
os.makedirs(file_path, exist_ok=True)
with open(f"{file_path}/class.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(class_labels)
ag_news_df.to_csv(f"{file_path}/master.csv")